In [1]:
import sys
sys.path+= ['/Users/ericxia/Documents/Research/iRF/irf/demo', '/Users/ericxia/anaconda/lib/python36.zip', '/Users/ericxia/anaconda/lib/python3.6', '/Users/ericxia/anaconda/lib/python3.6/lib-dynload', '/Users/ericxia/anaconda/lib/python3.6/site-packages', '/Users/ericxia/Documents/Research/iRF/irf']

In [2]:
import irf
from sklearn.datasets import load_boston, load_breast_cancer
import irf.ensemble.wrf as wrf
from irf.ensemble.wrf import wrf_reg
import numpy as np

from sklearn.model_selection import train_test_split

from irf import irf_utils
from irf import irf_jupyter_utils

/Users/ericxia/anaconda/lib/python3.6/site-packages/sklearn/externals/joblib/__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)
/Users/ericxia/anaconda/lib/python3.6/site-packages/sklearn/externals/six.py:31: DeprecationWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", DeprecationWarning)


In [3]:
sys.path

['/Library/Frameworks/Python.framework/Versions/3.6/lib/python36.zip',
 '/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6',
 '/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/lib-dynload',
 '',
 '/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages',
 '/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/IPython/extensions',
 '/Users/ericxia/.ipython',
 '/Users/ericxia/Documents/Research/iRF/irf/demo',
 '/Users/ericxia/anaconda/lib/python36.zip',
 '/Users/ericxia/anaconda/lib/python3.6',
 '/Users/ericxia/anaconda/lib/python3.6/lib-dynload',
 '/Users/ericxia/anaconda/lib/python3.6/site-packages',
 '/Users/ericxia/Documents/Research/iRF/irf']

In [4]:
from pyspark.ml.fpm import FPGrowth
from pyspark.sql import SparkSession
from pyspark.sql.types import *

In [5]:
data = load_breast_cancer()
X = data['data']
y = data['target']

In [6]:
X_train, X_test, y_train, y_test = train_test_split(
        data.data, data.target, train_size=0.9)

In [7]:
rf = wrf()
weight = np.ones((X.shape[1],)) / X.shape[1]
rf.fit(X, y, feature_weight=weight)
print(rf.feature_importances_)

[0.         0.02262965 0.         0.00306081 0.         0.
 0.00073711 0.         0.         0.         0.00235136 0.
 0.         0.00248438 0.         0.         0.         0.
 0.         0.         0.09398526 0.03078169 0.41096837 0.0961524
 0.00192235 0.         0.00065334 0.3342733  0.         0.        ]


In [8]:
all_rf_tree_data = irf_utils.get_rf_tree_data(rf=rf, X_train=X_train, X_test=X_test, y_test=y_test)

In [9]:
def convert(leaf_node_paths, feature_paths):
    leafnode_to_features = {}
    for i in range(len(leaf_node_paths)):
        leaf_node = leaf_node_paths[i][-1]
        leafnode_to_features[leaf_node] = feature_paths[i]
    return leafnode_to_features

In [10]:
interactions = np.array([])
inp = X_train[y_train==1]

for i in range(len(rf.estimators_)):
    key = "dtree" + str(i)
    tree_dict = all_rf_tree_data[key]
    #assume that ith entry of feature_paths corresponds to ith entry of leaf_node_paths
    feature_paths = tree_dict["all_uniq_leaf_paths_features"]
    leaf_nodes = tree_dict["all_leaf_nodes"]
    leaf_node_paths = tree_dict["all_leaf_node_paths"]
    decision_tree = rf.estimators_[i]
    
    end = decision_tree.apply(inp)
    leafnode_to_features = convert(leaf_node_paths, feature_paths)
    temp_features = np.vectorize(leafnode_to_features.get, otypes=[np.ndarray])(end)
    interactions = np.concatenate((interactions, temp_features))
    


In [11]:
#schema = StructType([StructField("id", IntegerType(), True), 
#                     StructField("list", ArrayType(), True)])

spark = SparkSession \
    .builder \
    .appName("iterative Random Forests with FP-Growth") \
    .getOrCreate()
    
input_list = [(i, interactions[i].tolist()) for i in range(len(interactions))]

df = spark.createDataFrame(input_list, ["id", "items"])
df.show()

+---+-------------------+
| id|              items|
+---+-------------------+
|  0|[1, 13, 20, 22, 27]|
|  1|[1, 13, 20, 22, 27]|
|  2|[1, 13, 20, 22, 27]|
|  3|[1, 13, 20, 22, 27]|
|  4|[1, 13, 20, 22, 27]|
|  5|[1, 13, 20, 22, 27]|
|  6|[1, 13, 20, 22, 27]|
|  7|[1, 13, 20, 22, 27]|
|  8|[1, 13, 20, 22, 27]|
|  9|[1, 13, 20, 22, 27]|
| 10|[1, 13, 20, 22, 27]|
| 11|[1, 13, 20, 22, 27]|
| 12|           [21, 22]|
| 13|[1, 13, 20, 22, 27]|
| 14|           [21, 22]|
| 15|[1, 13, 20, 22, 27]|
| 16|[1, 13, 20, 22, 27]|
| 17|[1, 13, 20, 22, 27]|
| 18|[1, 13, 20, 22, 27]|
| 19|   [13, 22, 23, 27]|
+---+-------------------+
only showing top 20 rows



In [12]:
fpGrowth = FPGrowth(itemsCol="items", minSupport=0.1, minConfidence=0.6)
model = fpGrowth.fit(df)

model.freqItemsets.show(40, False)

+----------------+----+
|items           |freq|
+----------------+----+
|[24]            |363 |
|[24, 22]        |363 |
|[24, 22, 27]    |363 |
|[24, 27]        |363 |
|[21]            |757 |
|[21, 20]        |354 |
|[21, 20, 27]    |353 |
|[21, 22]        |429 |
|[21, 22, 27]    |397 |
|[21, 27]        |725 |
|[27]            |3122|
|[1]             |651 |
|[1, 20]         |597 |
|[1, 20, 27]     |597 |
|[1, 22]         |335 |
|[1, 22, 27]     |335 |
|[1, 27]         |651 |
|[22]            |2055|
|[22, 27]        |2007|
|[20]            |1854|
|[20, 22]        |1063|
|[20, 22, 27]    |1062|
|[20, 27]        |1853|
|[13]            |594 |
|[13, 20]        |592 |
|[13, 20, 27]    |592 |
|[13, 27]        |594 |
|[10]            |553 |
|[10, 22]        |553 |
|[10, 22, 27]    |553 |
|[10, 27]        |553 |
|[23]            |1826|
|[23, 20]        |1199|
|[23, 20, 22]    |724 |
|[23, 20, 22, 27]|724 |
|[23, 20, 27]    |1199|
|[23, 22]        |1027|
|[23, 22, 27]    |1019|
|[23, 27]       

In [13]:
from prefixspan import PrefixSpan

In [16]:
inter = [interactions[i].tolist() for i in range(len(interactions))]
ps = PrefixSpan(inter)
ps.topk(100)

[(3122, [27]),
 (2055, [22]),
 (2007, [22, 27]),
 (1854, [20]),
 (1853, [20, 27]),
 (1826, [23]),
 (1818, [23, 27]),
 (1199, [20, 23]),
 (1199, [20, 23, 27]),
 (1063, [20, 22]),
 (1062, [20, 22, 27]),
 (1027, [22, 23]),
 (1019, [22, 23, 27]),
 (757, [21]),
 (725, [21, 27]),
 (724, [20, 22, 23]),
 (724, [20, 22, 23, 27]),
 (651, [1]),
 (651, [1, 27]),
 (597, [1, 20]),
 (597, [1, 20, 27]),
 (594, [13]),
 (594, [13, 27]),
 (592, [13, 20]),
 (592, [13, 20, 27]),
 (553, [10]),
 (553, [10, 22]),
 (553, [10, 22, 27]),
 (553, [10, 27]),
 (429, [21, 22]),
 (397, [21, 22, 27]),
 (363, [22, 24]),
 (363, [22, 24, 27]),
 (363, [24]),
 (363, [24, 27]),
 (354, [20, 21]),
 (353, [20, 21, 27]),
 (335, [1, 22]),
 (335, [1, 22, 27]),
 (310, [1, 21]),
 (310, [1, 21, 27]),
 (309, [13, 23]),
 (309, [13, 23, 27]),
 (307, [13, 20, 23]),
 (307, [13, 20, 23, 27]),
 (297, [3]),
 (297, [3, 22]),
 (294, [3, 22, 23]),
 (294, [3, 23]),
 (293, [1, 20, 21]),
 (293, [1, 20, 21, 27]),
 (291, [1, 20, 22]),
 (291, [1, 20,

[[1, 13, 20, 22, 27],
 [1, 13, 20, 22, 27],
 [1, 13, 20, 22, 27],
 [1, 13, 20, 22, 27],
 [1, 13, 20, 22, 27],
 [1, 13, 20, 22, 27],
 [1, 13, 20, 22, 27],
 [1, 13, 20, 22, 27],
 [1, 13, 20, 22, 27],
 [1, 13, 20, 22, 27],
 [1, 13, 20, 22, 27],
 [1, 13, 20, 22, 27],
 [21, 22],
 [1, 13, 20, 22, 27],
 [21, 22],
 [1, 13, 20, 22, 27],
 [1, 13, 20, 22, 27],
 [1, 13, 20, 22, 27],
 [1, 13, 20, 22, 27],
 [13, 22, 23, 27],
 [1, 13, 20, 22, 27],
 [1, 13, 20, 22, 27],
 [21, 22],
 [13, 20, 22, 27],
 [1, 13, 20, 22, 27],
 [1, 13, 20, 22, 27],
 [1, 13, 20, 22, 27],
 [1, 13, 20, 22, 27],
 [21, 22, 27],
 [1, 13, 20, 22, 27],
 [1, 13, 20, 22, 27],
 [1, 13, 20, 22, 27],
 [1, 13, 20, 22, 27],
 [1, 13, 20, 22, 27],
 [20, 21, 22, 27],
 [1, 13, 20, 22, 27],
 [1, 13, 20, 22, 27],
 [1, 13, 20, 22, 27],
 [1, 13, 20, 22, 27],
 [1, 13, 20, 22, 27],
 [1, 13, 20, 22, 27],
 [1, 13, 20, 22, 27],
 [1, 13, 20, 22, 27],
 [13, 20, 22, 27],
 [1, 13, 20, 22, 27],
 [1, 13, 20, 22, 27],
 [1, 13, 20, 22, 27],
 [21, 22],
 [21, 2